In [64]:
import pandas as pd
import numpy as np
import jieba.analyse
GOODS_EXCEL_PATH = 'taobao_goods.xlsx'
GOODS_STANDARD_EXCEL_PATH = 'taobao_goods_standard.xlsx'
# 清洗词
STOP_WORDS_FILE_PATH = 'stop_words.txt'
from pyecharts import options as opts
from pyecharts.globals import SymbolType
from pyecharts.charts import Pie, Bar, Map, WordCloud

In [65]:
data = pd.read_excel(GOODS_STANDARD_EXCEL_PATH)

In [6]:
jieba.analyse.set_stop_words(STOP_WORDS_FILE_PATH)

In [7]:
keywords_count_list = jieba.analyse.textrank(' '.join(data.title), topK=50, withWeight=True)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\sunbiao\AppData\Local\Temp\jieba.cache
Loading model cost 0.641 seconds.
Prefix dict has been built succesfully.


In [8]:
word_cloud = (
        WordCloud()
        .add("", keywords_count_list, word_size_range=[20, 100], shape=SymbolType.DIAMOND)
        .set_global_opts(title_opts=opts.TitleOpts('title="避孕套功能词云TOP50"'))

)

In [9]:
keywords_count_dict = {i[0]: 0 for i in reversed(keywords_count_list[:20])}

In [81]:
keywords_count_dict = {i[0]: 0 for i in keywords_count_list[:20]}

In [11]:
# 设置切分区域
price_list_bins = [0, 20, 40, 60, 80, 100, 120, 150, 200, 1000000]
# 设置切分后对应标签
price_list_labels = ['0-20', '21-40', '41-60', '61-80', '81-100', '101-120', '121-150', '151-200', '200以上']
data_labels_list = pd.cut(data['price'], bins=price_list_bins, labels=price_list_labels, include_lowest=True)
# 生成一个以listLabels为顺序的字典，这样就不需要后面重新排序
data_count = {i: 0 for i in price_list_labels}
# 统计结果
for value in data_labels_list:
    # get(value, num)函数的作用是获取字典中value对应的键值, num=0指示初始值大小。
    data_count[value] = data_count.get(value) + 1
bar = (
    Bar()
    .add_xaxis(list(data_count.keys()))
    .add_yaxis("", list(data_count.values()))
    .set_global_opts(
        title_opts=opts.TitleOpts(title="避孕套商品价格区间分布柱状体"),
        yaxis_opts=opts.AxisOpts(name="个商品"),
        xaxis_opts=opts.AxisOpts(name="商品售价：元")
    )
)
bar.render('price-bar.html')
# 生成饼图
age_count_list = [list(z) for z in zip(data_count.keys(), data_count.values())]
pie = (
    Pie()
    .add("", age_count_list)
    .set_global_opts(title_opts=opts.TitleOpts(title="避孕套商品价格区间饼图"))
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}"))
)
pie.render('price-pie.html')

'E:\\deep_learning\\taobao_spider\\price-pie.html'

In [41]:
from pyecharts.render import make_snapshot
from snapshot_selenium import snapshot

# 取前20高频的关键词
keywords_count_dict = {i[0]: 0 for i in reversed(keywords_count_list[:20])}
cut_words = jieba.cut(' '.join(data['title']))
for word in cut_words:
    for keyword in keywords_count_dict.keys():
        if word == keyword:
            keywords_count_dict[keyword] += 1
keywords_list = list(keywords_count_dict.items())
keywords_list.sort(key=lambda x: x[1], reverse=False)
keywords_count_dict = dict(keywords_list)
# 2.2生成柱状图
keywords_count_bar = (
    Bar()
    .add_xaxis(list(keywords_count_dict.keys()))
    .add_yaxis("heihei", list(keywords_count_dict.values()))
    .reversal_axis()
    .set_series_opts(label_opts=opts.LabelOpts(position="right"))
    .set_global_opts(
        title_opts=opts.TitleOpts(title="避孕套功能TOP20"),
        yaxis_opts=opts.AxisOpts(name="功能"),
        xaxis_opts=opts.AxisOpts(name="商品数")
    )
)
make_snapshot(snapshot, keywords_count_bar.render(), "bar.png")

In [38]:
from pyecharts.faker import Faker
from pyecharts import options as opts
from pyecharts.charts import Bar


c = (
    Bar()
    .add_xaxis(Faker.choose())
    .add_yaxis("商家A", Faker.values())
    .add_yaxis("商家B", Faker.values())
#         .set_series_opts(label_opts=opts.LabelOpts())
    .set_global_opts(
        title_opts=opts.TitleOpts(title="Bar-基本示例", subtitle="我是副标题")
    )
)
c.render()

'E:\\deep_learning\\taobao_spider\\render.html'

In [37]:
bar_base()

'E:\\deep_learning\\taobao_spider\\c.png'

In [45]:
province_sales = data.location.value_counts()

In [47]:
province_sales_list = [list(item) for item in province_sales.items()]

In [104]:
province_sales_map = (
        Map()
            .add("前两千款避孕套商家数量全国分布图", province_sales_dict, "china")
            .set_global_opts(
            visualmap_opts=opts.VisualMapOpts(max_=647),
        )
    )
province_sales_map.render('province-seller-map.html')

'E:\\deep_learning\\taobao_spider\\province-seller-map.html'

In [99]:
item = {}
for location in list(set(data['location'].values)):
    for row in data.iterrows():
        if location == row[1].location:
            item.setdefault(location, []).append(row[1].sales)

for location in item:
    item[location] = round(sum(item[location])/len(item[location]), 1)

list1 = list(item.items())
list1.sort(key=lambda x: x[1], reverse=False)
item = dict(list1)
k = (
    Bar()
    .add_xaxis(list(item.keys()))
    .add_yaxis("", list(item.values()))
    .reversal_axis()
    .set_series_opts(label_opts=opts.LabelOpts(position='right'))
    .set_global_opts(
        title_opts=opts.TitleOpts(title='heihei'),
        yaxis_opts=opts.AxisOpts(name='y'),
        xaxis_opts=opts.AxisOpts(name='x')
    )
)
k.render('1.html')
p = (
    Pie()
    .add("", list(item.items()))
    .set_global_opts(title_opts=opts.TitleOpts(title='商品'))
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}"))
)
p.render('2.html')

'E:\\deep_learning\\taobao_spider\\2.html'

In [114]:
data=data.pivot_table(index='location', values='sales', aggfunc=np.mean)
data.sort_values('sales', inplace=True, ascending=False)
data

,sales
location,
山西,1535.793103
广东,804.899536
上海,713.702760
日本,658.444444
北京,495.701389
天津,396.095238
江苏,390.172932
安徽,366.642857
湖北,336.625000


In [118]:
# 设置切分区域
listBins = [0, 1000, 5000, 10000, 50000, 100000, 1000000]
# 设置切分后对应标签
listLabels = ['一千以内', '一千到五千', '五千到一万', '一万到五万', '五万到十万', '十万以上']
sales_count_list = pd.cut(data['sales'],
                          bins=listBins,
                          labels=listLabels,

)
# 生成柱状图
bar = (
    Bar()
    .add_xaxis(list(sales_count_list.keys()))
    .add_yaxis("", list(sales_count_list.values()))
    .set_global_opts(
        title_opts=opts.TitleOpts(title="避孕套商品销量区间分布柱状图"),
        yaxis_opts=opts.AxisOpts(name="个商品"),
        xaxis_opts=opts.AxisOpts(name="销售件数")
    )
)
bar.render('sales-bar.html')

TypeError: 'Categorical' object is not callable